# HAI Deepfake - Ultimate Inference Pipeline (Top-K Strategy)

**전략**: 
1. 로컬 전처리 데이터 (RetinaFace Crop) 사용
2. EfficientNet-V2-M (480x480) 순정/학습 모델 자동 전환
3. **Top-K Average**: 비디오 30프레임 중 '가짜' 확률이 높은 상위 5장의 평균값 사용

In [ ]:
# 1. 환경 설정 및 필수 라이브러리 설치
!nvidia-smi

from google.colab import drive
drive.mount('/content/drive')

print("Install Dependencies...")
# Pillow 업그레이드 포함 (ImportError 방지)
!pip install -q timm facenet-pytorch albumentations pillow --upgrade

In [ ]:
# 1.5. 코드 동기화 (Git Sync) - src 폴더 가져오기
import os

REPO_URL = "https://github.com/CBottle/HAI_Deepfake.git"
PROJECT_DIR = "/content/HAI_Deepfake"
BRANCH = "HK" 

if os.path.exists(PROJECT_DIR):
    print(f"Pulling latest code from {BRANCH}...")
    %cd {PROJECT_DIR}
    !git fetch --all
    !git reset --hard origin/{BRANCH}
else:
    print(f"Cloning project ({BRANCH})...")
    !git clone -b {BRANCH} {REPO_URL} {PROJECT_DIR}
    %cd {PROJECT_DIR}

print("✅ 코드 준비 완료!")

In [ ]:
import os
import shutil

# 2. 데이터 준비
DRIVE_ZIP_PATH = '/content/drive/MyDrive/HAI_Deepfake/processed_test_data.zip'
LOCAL_DATA_DIR = '/content/processed_test_data'

if not os.path.exists(LOCAL_DATA_DIR):
    print("📂 데이터 복사 및 압축 해제 중...")
    os.makedirs(LOCAL_DATA_DIR, exist_ok=True)
    !cp {DRIVE_ZIP_PATH} /content/data.zip
    !unzip -q /content/data.zip -d {LOCAL_DATA_DIR}
    print("✅ 데이터 준비 완료!")
else:
    print("✅ 데이터가 이미 존재합니다.")

# 모델 경로
MODEL_PATH = '/content/drive/MyDrive/HAI_Deepfake/checkpoints/best_model.pt'

In [ ]:
# 3. 추론 코드 실행 (Inference Engine)
import os
import sys
import torch
import timm
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F
from timm.data import resolve_data_config, create_transform

# src 모듈 경로 추가
sys.path.append("/content/HAI_Deepfake")
try:
    from src.models import load_model
    print("✅ src.models loaded successfully!")
except ImportError:
    print("❌ Failed to load src.models. Please check Step 1.5.")

# 설정값 재정의
MODEL_NAME = "tf_efficientnetv2_m.in21k"
MODEL_PATH = '/content/drive/MyDrive/HAI_Deepfake/checkpoints/best_model.pt'
LOCAL_DATA_DIR = '/content/processed_test_data'
OUTPUT_CSV = "submission.csv"

class ProcessedDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.items = sorted([item for item in self.data_dir.glob('*') if not item.name.startswith('.')])
        
        if len(self.items) == 1 and self.items[0].is_dir():
            print(f"🔍 하위 폴더 감지: {self.items[0].name} 로 진입합니다.")
            self.data_dir = self.items[0]
            self.items = sorted([item for item in self.data_dir.glob('*') if not item.name.startswith('.')])

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item_path = self.items[idx]
        pixel_values_list = []
        
        if item_path.is_dir():
            img_paths = sorted(list(item_path.glob('*.jpg')))
            for p in img_paths:
                img = Image.open(p).convert("RGB")
                if self.transform:
                    pixel_values_list.append(self.transform(img))
        else:
            img = Image.open(item_path).convert("RGB")
            if self.transform:
                pixel_values_list.append(self.transform(img))

        if not pixel_values_list:
            return torch.zeros(1, 3, 480, 480), item_path.name

        return torch.stack(pixel_values_list), item_path.name

def run_inference():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # 모델 로드 (분기 처리)
    if os.path.exists(MODEL_PATH):
        print(f"✅ Loading Fine-tuned Model (6-channel SRM): {MODEL_PATH}")
        # 6채널 DeepfakeDetector 로드
        model = load_model(MODEL_PATH, MODEL_NAME, device)
        default_cfg = model.model.default_cfg
    else:
        print(f"⚠️ Loading Pure Pretrained Model (3-channel): {MODEL_NAME}")
        # 3채널 순정 모델 로드
        model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=2).to(device)
        default_cfg = model.default_cfg
    
    model.eval()

    # Transform
    data_config = resolve_data_config(default_cfg, model=model)
    print(f"Data Config: {data_config}")
    transform = create_transform(**data_config)

    # DataLoader
    dataset = ProcessedDataset(LOCAL_DATA_DIR, transform=transform)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

    # 추론
    results = {}
    print(f"🚀 Starting inference on {len(dataset)} items...")
    
    with torch.no_grad():
        for pixel_values, filenames in tqdm(dataloader):
            pixel_values = pixel_values.squeeze(0).to(device)
            filename = filenames[0]
            
            with torch.cuda.amp.autocast():
                outputs = model(pixel_values)
                if hasattr(outputs, 'logits'):
                    logits = outputs.logits
                else:
                    logits = outputs
                
                probs = F.softmax(logits, dim=1)[:, 1] # Fake 확률
            
            k = min(5, len(probs))
            top_k_probs, _ = torch.topk(probs, k)
            final_prob = float(top_k_probs.mean().cpu().item())
            
            clean_filename = filename.replace('_frames', '')
            results[clean_filename] = final_prob

    return results

results = run_inference()

submission = pd.DataFrame(list(results.items()), columns=['filename', 'prob'])
submission.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Submission saved: {OUTPUT_CSV}")

BACKUP_PATH = '/content/drive/MyDrive/HAI_Deepfake/submission_final_srm.csv'
!cp {OUTPUT_CSV} {BACKUP_PATH}
print(f"💾 Drive Backup: {BACKUP_PATH}")